In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
import sys
sys.path.append('/Users/dbao/Downloads/research/rc_scripts/analysis')

from final_analysis.process_helper.df_add_features import df_add_features
from final_analysis.process_helper.df_undo_start_end import df_undo_start_end
from final_analysis.process_helper.df_undo_seq import df_undo_seq

# Load data

In [3]:
home_dir = '/Users/dbao/My_Drive'+'/road_construction/data/2022_online/'
data_dir = home_dir + 'R_analysis_data/'
out_dir = home_dir + 'figures/figures_final/'
out_dir2 = home_dir + 'stat_model/final/'

In [4]:
data_choice_level = pd.read_csv(data_dir +  'choice_level/choicelevel_data.csv') # first column is index  
data_choice_level = data_choice_level.rename(columns={'Unnamed: 0':'index_copy'})
data_choice_level = df_add_features(data_choice_level)
sc_data_choice_level = data_choice_level[data_choice_level['condition']==1].reset_index(drop=True) 

## process for undo sequence

In [5]:
condition = f"(lastUndo == 1)"
accu_error = df_undo_start_end(sc_data_choice_level, condition)

In [13]:
accu_error

,subjects,puzzleID,cum_error_beginning,cum_error_end,n_connect_beginning,n_connect_end,undo_length,mas_gain,error_beginning,sequential_single,terminal_beginning,category
3,0,1,1,0,2,1,1,1,1,True,0,1
2,0,3,0,0,1,0,1,0,0,True,0,0
8,0,7,1,0,7,6,1,1,1,True,1,1
5,0,10,1,0,4,3,1,1,1,True,0,1
4,0,12,1,1,3,2,1,0,0,True,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
40,99,42,0,0,8,6,2,0,0,False,1,0
14,99,43,1,0,10,6,4,1,0,False,1,1
9,99,45,1,1,8,7,1,0,0,True,1,2
14,99,45,1,0,8,4,4,1,0,False,1,1


# get test statistic

In [18]:
avg_sub_masgain = accu_error.groupby(['subjects'])['mas_gain'].mean().mean()
avg_sub_masgain

1.018126284220514

# get undo length distribution for each subject

In [7]:
undo_length_map = accu_error.groupby(['subjects','n_connect_beginning'])['undo_length'].apply(list).reset_index()
undo_length_map[undo_length_map.subjects == 2]

,subjects,n_connect_beginning,undo_length
6,2,1,[1]
7,2,4,"[1, 4, 3, 4, 1]"
8,2,5,"[3, 2, 5, 4, 5, 4]"
9,2,6,"[6, 6]"
10,2,7,"[1, 2]"
11,2,8,[8]


# get df for undo sequence

In [8]:
condition = f"(firstUndo == 1)"
undo_seqs = df_undo_seq(sc_data_choice_level, condition)

In [9]:
undo_seqs[undo_seqs['subjects']==2]

,subjects,puzzleID,n_connect_beginning,path_list,currMAS_list
0,2,5,6.0,"[0, 12, 23, 24, 17, 5, 4]","[8, 8, 7, 7, 7, 7, 7]"
0,2,8,5.0,"[0, 5, 14, 17, 28, 11]","[8, 8, 8, 8, 7, 6]"
0,2,12,4.0,"[0, 16, 22, 4, 19]","[8, 7, 7, 7, 5]"
0,2,14,6.0,"[0, 7, 25, 19, 8, 18, 1]","[9, 6, 6, 6, 6, 6, 6]"
0,2,17,5.0,"[0, 9, 16, 17, 8, 2]","[7, 7, 7, 6, 6, 5]"
0,2,19,5.0,"[0, 9, 6, 14, 4, 28]","[7, 6, 6, 6, 5, 5]"
1,2,19,4.0,"[0, 25, 23, 8, 3]","[7, 6, 4, 4, 4]"
0,2,20,5.0,"[0, 1, 8, 23, 9, 6]","[8, 7, 5, 6, 6, 6]"
1,2,20,4.0,"[0, 1, 27, 21, 8]","[8, 7, 5, 4, 6]"
0,2,22,4.0,"[0, 13, 15, 20, 24]","[8, 7, 7, 7, 7]"


# simulate for null distribution

In [32]:
# for each row of error_map, randomly sample 1 undo_length from undo_length_map
def get_sim_avg_mas_gain(undo_seqs, undo_length_map):
    results = []
    for i in range(len(undo_seqs)):
        subset = undo_seqs.iloc[i]
        n_connect_beginning = subset['n_connect_beginning']
        subjects = subset['subjects']

        filterd = undo_length_map[(undo_length_map['subjects']==subjects) & (undo_length_map['n_connect_beginning']==n_connect_beginning)]
        sampled_undo_length = random.choice(filterd['undo_length'].values[0])
    
        currMAS_list = subset['currMAS_list']
        sampled_mas_gain = currMAS_list[-sampled_undo_length-1] - currMAS_list[-1]
        results.append({'subjects': subjects, 'n_connect_beginning': n_connect_beginning, 'mas_gain': sampled_mas_gain})
    sampled_mas_gains = pd.DataFrame(results)
    avg = sampled_mas_gains.groupby(['subjects'])['mas_gain'].mean().mean()
    return avg

In [33]:
get_sim_avg_mas_gain(undo_seqs, undo_length_map)

0.8518073289770138

In [34]:
sim_list = []
for i in range(1000):
    sim_list.append(get_sim_avg_mas_gain(undo_seqs, undo_length_map))

KeyboardInterrupt: 

In [36]:
# get the position of avg_sub_masgain in sim_list's quantile
sorted_list = sorted(sim_list)
# Get the position of the target in the sorted list
position = len([x for x in sorted_list if x < avg_sub_masgain])
print(len(sorted_list))
position/len(sorted_list)

290


1.0